In [ ]:
import sys, warnings
sys.path.append('..')
warnings.filterwarnings('ignore')

from pathlib import Path
from dataclasses import dataclass

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import trange

import torch
from diffusers import StableDiffusionImg2ImgPipeline

from arch import utils

In [ ]:
DEVICE = utils.device_mapper()
print(f"Device: {str(DEVICE).upper()}")

In [ ]:
pipeline = StableDiffusionImg2ImgPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", variant='fp16', torch_dtype=torch.float16)
pipeline = pipeline.to(DEVICE)

In [ ]:
PATH = Path('data/puppy.png')
init_image = utils.load_image(PATH)
init_image

---

In [ ]:
def transform(img: Image.Image) -> torch.Tensor:
    # Resize to factor of 32
    w, h = map(lambda x: x - x % 32, img.size)
    img = img.resize((w, h), resample=Image.LANCZOS)

    img = utils.image2array(img, dtype=np.float16)
    img = img * 2.0 - 1.0
    img = img[None].transpose(0, 3, 1, 2)
    return torch.from_numpy(img)

In [ ]:
def pgd_attack(x, model, eps=0.1, step_size=0.015, iters=40):
    x_adv = x.clone().detach() + (torch.rand_like(x) * 2 * eps - eps)

    for i in (t := trange(iters)):
        actual_step_size = step_size * (1 - i / iters / 100)

        x_adv.requires_grad = True

        loss = model(x_adv).latent_dist.mean.norm()
        grad = torch.autograd.grad(loss, [x_adv])[0]
        x_adv = x_adv.detach() - grad.sign() * actual_step_size

        x_adv = torch.clamp(x_adv, x-eps, x+eps)
        x_adv = torch.clamp(x_adv, -1, 1)

        t.set_description(f"[Running attack]: Loss {loss.item():.2f} | Actual Step: {actual_step_size:.4f}")

    return x_adv

In [ ]:
# eps | Higher = Less imperceptible (0.06)
# step | Set smaller than eps (0.02)
# iters | Higher = Stronger (1000)

x = transform(init_image).to(DEVICE)
adv_x = pgd_attack(x, model=pipeline.vae.encode, eps=0.06, step_size=0.02, iters=1000)

In [ ]:
adv_image = (adv_x / 2 + 0.5).clamp(0, 1)
adv_image = adv_image.cpu().numpy().transpose(0, 2, 3, 1)[0]
adv_image = utils.array2image(adv_image)
adv_image

---

In [ ]:
@dataclass
class HyperConfig:
    prompt = "a photograph of a dog under heavy rain on muddy ground"
    strength = 0.5
    cfg_scale = 7.0
    infer_steps = 50

config = HyperConfig()

In [ ]:
image_nat = pipeline(prompt=config.prompt, 
                        image=init_image, 
                        strength=config.strength, 
                        guidance_scale=config.cfg_scale, 
                        num_inference_steps=config.infer_steps).images[0]

image_adv = pipeline(prompt=config.prompt, 
                        image=adv_image, 
                        strength=config.strength, 
                        guidance_scale=config.cfg_scale, 
                        num_inference_steps=config.infer_steps).images[0]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(20,6))

ax[0].imshow(init_image)
ax[1].imshow(adv_image)
ax[2].imshow(image_nat)
ax[3].imshow(image_adv)

ax[0].set_title('Source Image', fontsize=12)
ax[1].set_title('Adv Image', fontsize=12)
ax[2].set_title('Gen. Image Nat.', fontsize=12)
ax[3].set_title('Gen. Image Adv.', fontsize=12)

for i in range(4):
    ax[i].grid(False), ax[i].axis('off')

fig.tight_layout()